In [121]:
import numpy as np
from collections import ChainMap
import pickle

In [122]:
# univeral settings common to ALL runs
uni_settings = []
for p_switch in [0.1]:
    for p_wind in [0.1, 0.2]:
        
        # set p_wind_i and p_wind_j
        p_wind_i, p_wind_j = p_wind, p_wind
        
        for allow_stay_action in [True, False]:
            for num_cycles in [None]:
                for max_iters in [50000]:
                    for alpha in [0.1, 1.0]:
                        for gamma in [1.0, 0.5, 0.0]:
                            for epsilon in [0.0, 0.05]:
                                for verbose in [False]:
                                    for river_restart in [True, False]:

                                        # create a dictionary containing these universal settings
                                        uni_setting = {"p_switch" : p_switch, 
                                                       "p_wind_i" : p_wind_i, 
                                                       "p_wind_j" : p_wind_j,
                                                       "allow_stay_action" : allow_stay_action,
                                                       "num_cycles" : num_cycles,
                                                       "max_iters" : max_iters,
                                                       "alpha" : alpha,
                                                       "gamma" : gamma,
                                                       "epsilon" : epsilon,
                                                       "verbose" : verbose,
                                                       "river_restart" : river_restart}    

                                        # add to our set of all universal settings
                                        uni_settings.append(uni_setting.copy())

In [123]:
# imputer-specific settings
imp_settings = []
for impute_method in ["joint", "joint-conservative", 
                      "random_action", "last_fobs", "last_obs", "missing_state"]:
    
    # what are our options for K? Iterate thru them.
    Ks = [1] if impute_method not in ["joint", "joint-conservative"] else [1, 5, 10]
    for K in Ks:
        
        # what are our possible action options? Iterate thru those too.
        action_options = ["voting1"] if K == 1 else ["voting1", "voting2", "averaging"]
        for action_option in action_options:
            
            # these are our imputer specific settings
            imp_setting = {"impute_method" : impute_method, 
                           "action_option" : action_option, 
                           "K" : K}
            
            # add to our set of imputer-specific settings
            imp_settings.append(imp_setting.copy())

In [124]:
# environment-specific settings
env_settings = []
for env_missing in ["MCAR", "Mcolor", "Mfog"]:
    
    # MCAR_thetas only makes sense for MCAR! Iterate.
    if env_missing == "MCAR":
        MCAR_thetas = [np.full(3, t) for t in (list(np.round(np.arange(0.0, 1.0, 0.1), 2) ))]
    else:
        MCAR_thetas = [np.array([None, None, None])]
    for MCAR_theta in MCAR_thetas:
        
        # theta_in and theta_out only make sense for MFOG
        if env_missing == "Mfog":
            thetas_IO = [(np.array([0.5, 0.5, 0.5]), np.array([0.2, 0.2, 0.2])),
                         (np.array([0.8, 0.8, 0.8]), np.array([0.3, 0.3, 0.3]))]
        else:
            thetas_IO = [(np.array([None, None, None]), np.array([None, None, None]))]
        for theta_in, theta_out in thetas_IO:
            
            # finally deal with color_theta_dict
            if env_missing == "Mcolor":
                color_theta_dicts = [{0 : np.array([0.1, 0.1, 0.1]),
                                      1 : np.array([0.2, 0.2, 0.2]),
                                      2 : np.array([0.3, 0.3, 0.3])}, #MAR - easier
                                     {0 : np.array([0.1, 0.1, 0.0]),
                                      1 : np.array([0.2, 0.2, 0.0]),
                                      2 : np.array([0.3, 0.3, 0.0])}, #NMAR - easier
                                     {0 : np.array([0.2, 0.2, 0.0]),
                                      1 : np.array([0.4, 0.4, 0.0]),
                                      2 : np.array([0.6, 0.6, 0.0])}, #MAR - harder
                                     {0 : np.array([0.2, 0.2, 0.2]),
                                      1 : np.array([0.4, 0.4, 0.4]),
                                      2 : np.array([0.6, 0.6, 0.6])},] #NMAR - harder
            else:
                color_theta_dicts = [{0 : np.array([None, None, None]), 
                                      1 : np.array([None, None, None]), 
                                      2 : np.array([None, None, None])}]
            for color_theta_dict in color_theta_dicts:
                
                # assemble our env-specific dictionary + add to our list
                env_setting = {"env_missing" : env_missing,
                               "MCAR_theta" : MCAR_theta,
                               "theta_in" : theta_in,
                               "theta_out" : theta_out,
                               "color_theta_dict" : color_theta_dict}
                env_settings.append(env_setting.copy())
            
# seed settings
seed_settings = list(range(5))

In [125]:
# combine uni_settings with imp_settings: we'll deal with env_settings and seed_settings within the script.
combined_settings = []
for uni_setting in uni_settings:
    for imp_setting in imp_settings:
        combined_settings.append(dict(ChainMap(uni_setting.copy(), imp_setting.copy())))

In [126]:
# save combined settings (1728x of them) as a pickle
with open("combined_settings.pickle", "wb") as file:
    pickle.dump(combined_settings, file)

# save the 16x env settings too
with open("env_settings.pickle", "wb") as file:
    pickle.dump(env_settings, file)